In [ ]:
#| default_exp map

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *

In [ ]:
#| export
def load(json_file):
    """
    Load JSON data from a file.

    Args:
    - `json_file` (`str` or `Path`): Path to the JSON file.

    Returns:
    - `data` (`dict`): Parsed JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data

In [ ]:
#| export
def parse(filename):
    """
    Parse a filename to extract the surgery name, video name, and timestamp.

    Assumes filename format: `surgeryName_videoName_timestamp.extension`

    Args:
    - `filename` (`str` or `Path`): Filename to parse.

    Returns:
    - `surgery_name` (`str`): Extracted surgery name.
    - `video_name` (`str`): Extracted video name.
    - `timestamp` (`float`): Extracted timestamp.
    """
    # Remove the file extension
    base = Path(filename).stem
    # Split the base by underscores
    parts = base.split('_')
    if len(parts) < 3:
        raise ValueError(f"Filename '{filename}' does not contain enough underscores.")

    surgery_name = parts[0]
    timestamp_str = parts[-1]
    try:
        timestamp = float(timestamp_str)
    except ValueError:
        raise ValueError(f"Expected a timestamp at the end of '{filename}', but got '{timestamp_str}'.")

    video_name = '_'.join(parts[1:-1])

    return surgery_name, video_name, timestamp

In [ ]:
#| export
def extract(data, video_name, timestamp):
    """
    Extract labels for a given video and timestamp from the data.

    Args:
    - `data` (`list`): Annotations data.
    - `video_name` (`str`): Name of the video.
    - `timestamp` (`float`): Timestamp of the frame.

    Returns:
    - `labels` (`list`): List of labels corresponding to the timestamp.
    """
    labels = []
    for entry in data:
        file_upload = entry.get('file_upload', '')
        if video_name in file_upload:
            for annotation in entry.get('annotations', []):
                for result in annotation.get('result', []):
                    value = result.get('value', {})
                    start = value.get('start', 0)
                    end = value.get('end', 0)
                    if start <= timestamp <= end:
                        labels.extend(value.get('labels', []))
    return labels

In [ ]:
#| export
def map_labels(dataroot_path):
    """
    Map labels to frames based on annotations.

    Args:
    - `dataroot_path` (`str` or `Path`): Root path of the data directory.

    Returns:
    - `results` (`list` of `dict`): List containing filename and corresponding labels.
    """
    frames_folder = Path(dataroot_path) / 'frames'
    results = []
    # Retrieve all image files in frames_folder
    filenames = get_image_files(frames_folder)

    # Cache for loaded json data per surgery
    surgery_data_cache = {}

    for filepath in filenames:
        filename = filepath.name
        surgery_name, video_name, timestamp = parse(filename)

        # Load the json file for this surgery if not already loaded
        if surgery_name not in surgery_data_cache:
            json_file = Path(dataroot_path) / 'videos' / surgery_name / f'{surgery_name}.json'
            if not json_file.exists():
                print(f"Warning: JSON file '{json_file}' does not exist.")
                surgery_data_cache[surgery_name] = None
            else:
                data = load(json_file)
                surgery_data_cache[surgery_name] = data

        data = surgery_data_cache[surgery_name]

        if data is None:
            labels = []
        else:
            labels = extract(data, video_name, timestamp)

        labels_list = sorted(set(labels))
        timeline_label_raw = ','.join(labels_list)
        transition = len(labels_list) > 1
        timeline_label = labels_list[0] if labels_list else ''

        if timeline_label:
            label_parts = timeline_label.split('.')
            timeline_phase_label = label_parts[0] if len(label_parts) > 0 else ''
            timeline_task_label = label_parts[1] if len(label_parts) > 1 else ''
            timeline_action_label = label_parts[2] if len(label_parts) > 2 else ''
        else:
            timeline_phase_label = ''
            timeline_task_label = ''
            timeline_action_label = ''

        results.append({
            'filename': Path(filename).stem,
            'surgery_name': surgery_name,
            'video_name': video_name,
            'timestamp': timestamp,
            'timeline_label_raw': timeline_label_raw,
            'transition': transition,
            'timeline_label': timeline_label,
            'timeline_phase_label': timeline_phase_label,
            'timeline_task_label': timeline_task_label,
            'timeline_action_label': timeline_action_label
        })

    return results

In [ ]:
#| export
def save(results, dataroot_path):
    """
    Save the mapping results to a CSV file.

    Args:
    - `results` (`list` of `dict`): Results from mapping labels to frames.
    - `dataroot_path` (`str` or `Path`): Root path of the data directory.

    Returns:
    - `None`
    """
    df = pd.DataFrame(results)

    # Sort the DataFrame based on surgery_name, video_name, and timestamp
    df.sort_values(by=['surgery_name', 'video_name', 'timestamp'], inplace=True)

    # Find the two surgeries with the least number of frames
    least_frame_surgeries = df['surgery_name'].value_counts().nsmallest(2).index.tolist()

    # Add 'valid' column
    df['valid'] = df['surgery_name'].isin(least_frame_surgeries)

    csv_path = Path(dataroot_path) / 'timeline_labels.csv'
    df.to_csv(csv_path, index=False)

In [ ]:
#| export
def extract_labels(dataroot_path):
    """
    Extract labels for frames and save to 'timeline_labels.csv'.

    Args:
    - `dataroot_path` (`str` or `Path`): Root path of the data directory.

    Returns:
    - `None`
    """
    dataroot_path = Path(dataroot_path)
    results = map_labels(dataroot_path)
    save(results, dataroot_path)

In [9]:
#| export
def display_stats(dataroot_path):
    """
    Display basic statistics of the dataframe generated by extract_labels.

    Args:
    - `dataroot_path` (`str` or `Path`): Root path of the data directory.

    Returns:
    - `None`
    """
    dataroot_path = Path(dataroot_path)
    csv_path = dataroot_path / 'timeline_labels.csv'
    if not csv_path.exists():
        print(f"CSV file '{csv_path}' does not exist.")
        return

    df = pd.read_csv(csv_path)

    total_frames = len(df)
    transition_frames = df['transition'].sum()
    transition_percentage = (transition_frames / total_frames) * 100

    training_frames = df[~df['valid']].shape[0]
    validation_frames = df[df['valid']].shape[0]
    ratio_train_val = training_frames / validation_frames if validation_frames != 0 else float('inf')

    training_ratio = (training_frames / total_frames) * 100
    validation_ratio = (validation_frames / total_frames) * 100

    print(f"Total frames: {total_frames}")
    print(f"Transition frames: {transition_frames}")
    print(f"Transition percentage: {transition_percentage:.2f}%")
    print(f"Training frames: {training_frames} ({training_ratio:.2f}% of total)")
    print(f"Validation frames: {validation_frames} ({validation_ratio:.2f}% of total)")
    print(f"Training to validation ratio: {ratio_train_val:.2f}")

    # Prepare a table showing label and its proportion in valid and training set
    total_counts = df['timeline_label'].value_counts().reset_index()
    total_counts.columns = ['Label', 'Total_Count']
    total_counts['Total_Percent'] = (total_counts['Total_Count'] / total_frames) * 100

    train_counts = df[~df['valid']]['timeline_label'].value_counts().reset_index()
    train_counts.columns = ['Label', 'Train_Count']
    train_counts['Train_Percent'] = (train_counts['Train_Count'] / training_frames) * 100

    val_counts = df[df['valid']]['timeline_label'].value_counts().reset_index()
    val_counts.columns = ['Label', 'Valid_Count']
    val_counts['Valid_Percent'] = (val_counts['Valid_Count'] / validation_frames) * 100

    # Merge the counts
    counts_df = pd.merge(total_counts, train_counts, on='Label', how='outer')
    counts_df = pd.merge(counts_df, val_counts, on='Label', how='outer')
    counts_df.fillna(0, inplace=True)

    # Reorder columns
    counts_df = counts_df[['Label', 'Total_Count', 'Total_Percent', 'Train_Count', 'Train_Percent', 'Valid_Count', 'Valid_Percent']]

    # Round percentages
    counts_df['Total_Percent'] = counts_df['Total_Percent'].round(2)
    counts_df['Train_Percent'] = counts_df['Train_Percent'].round(2)
    counts_df['Valid_Percent'] = counts_df['Valid_Percent'].round(2)

    print("\nLabel distribution and proportions:")
    print(counts_df.to_string(index=False))

In [ ]:
# Example usage
extract_labels('../temset')

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()